In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from enum import Enum
from sklearn.datasets import load_iris
from typing import Callable, Iterable, List, Tuple

In [2]:
class HyperParams(Enum):
    ACTIVATION     = tf.nn.relu
    BATCH_SIZE     = 5
    EPOCHS         = 500
    HIDDEN_NEURONS = 10
    NORMALIZER     = tf.nn.softmax
    OUTPUT_NEURONS = 3
    OPTIMIZER      = tf.keras.optimizers.Adam

In [3]:
iris = load_iris()
xdat = iris.data
ydat = iris.target

In [ ]:
def to_tensor(self, depth: int = 3) -> None:
        self.xtrn = tf.convert_to_tensor(self.xtrn, dtype = np.float32) 
        self.xtst = tf.convert_to_tensor(self.xtst, dtype = np.float32)
        self.ytrn = tf.convert_to_tensor(tf.one_hot(self.ytrn, depth = depth))
        self.ytst = tf.convert_to_tensor(tf.one_hot(self.ytst, depth = depth))

In [20]:
class Data:
    def __init__(self,xdat: np.ndarray,ydat: np.ndarray,ratio: float = 0.3) -> Tuple:
        self.xdat = xdat
        self.ydat = ydat
        self.ratio = ratio
        
    def partition(self) -> None:
        scnt = self.xdat.shape[0] / np.unique(self.ydat).shape[0]
        ntst = int(self.xdat.shape[0] * self.ratio / (np.unique(self.ydat)).shape[0])
        idx  = np.random.choice(np.arange(0, self.ydat.shape[0] / np.unique(self.ydat).shape[0], dtype = int), ntst, replace = False)
        for i in np.arange(1, np.unique(self.ydat).shape[0]):
            idx = np.concatenate((idx, np.random.choice(np.arange((scnt * i), scnt * (i + 1), dtype = int), ntst, replace = False)))

        self.xtrn = self.xdat[np.where(~np.in1d(np.arange(0, self.ydat.shape[0]), idx))[0], :]
        self.ytrn = self.ydat[np.where(~np.in1d(np.arange(0, self.ydat.shape[0]), idx))[0]]
        self.xtst = self.xdat[idx, :]
        self.ytst = self.ydat[idx]
    
    def to_tensor(self, depth: int = 3) -> None:
        self.xtrn = tf.convert_to_tensor(self.xtrn, dtype=np.float32)
        self.xtst = tf.convert_to_tensor(self.xtst, dtype=np.float32)
        self.ytrn = tf.convert_to_tensor(tf.one_hot(self.ytrn, depth=depth))
        self.ytst = tf.convert_to_tensor(tf.one_hot(self.ytst, depth=depth))
        
    def batch(self, num: int = 16) -> None:
        try:
            size = self.xtrn.shape[0] / num
            if self.xtrn.shape[0] % num != 0:
                sizes = [tf.floor(size).numpy().astype(int) for i in range(num)] + [self.xtrn.shape[0] % num]
            else:
                sizes = [tf.floor(size).numpy().astype(int) for i in range(num)]

            self.xtrn_batches = tf.split(self.xtrn, num_or_size_splits = sizes, axis = 0)
            self.ytrn_batches = tf.split(self.ytrn, num_or_size_splits = sizes, axis = 0)

            num = int(self.xtst.shape[0] / sizes[0])
            if self.xtst.shape[0] % sizes[0] != 0:
                sizes = [sizes[i] for i in range(num)] + [self.xtst.shape[0] % sizes[0]]
            else:
                sizes = [sizes[i] for i in range(num)]

            self.xtst_batches = tf.split(self.xtst, num_or_size_splits = sizes, axis = 0)
            self.ytst_batches = tf.split(self.ytst, num_or_size_splits = sizes, axis = 0)
        except:
            self.xtrn_batches = [self.xtrn]
            self.ytrn_batches = [self.ytrn]
            self.xtst_batches = [self.xtst]
            self.ytst_batches = [self.ytst]

In [38]:
class Dense:
    def __init__(self, i: int, o: int, f: Callable[[tf.Tensor], tf.Tensor], initializer: Callable = tf.random.normal) -> None:
        self.w = tf.Variable(initializer([i, o]))
        self.b = tf.Variable(initializer([o]))
        self.f = f

    def __call__(self, x: tf.Tensor) -> tf.Tensor:
        if callable(self.f):
            return self.f(tf.add(tf.matmul(x, self.w), self.b))
        else:
            return tf.add(tf.matmul(x, self.w), self.b)

In [45]:
class Chain:

    def __init__(self, layers: List[Iterable[Dense]]) -> None:
        self.layers = layers
    
    def __call__(self, x: tf.Tensor) -> tf.Tensor:
        self.out = x; self.params = []
        for l in self.layers:
            self.out = l(self.out)
            self.params.append([l.w, l.b])
        
        self.params = [j for i in self.params for j in i]
        return self.out

    def backward(self, inputs: tf.Tensor, targets: tf.Tensor) -> None:
        grads = self.grad(inputs, targets)
        self.optimize(grads, 0.001)
    
    def loss(self, preds: tf.Tensor, targets: tf.Tensor) -> tf.Tensor:
        return tf.reduce_mean(
            tf.keras.losses.categorical_crossentropy(
                targets, preds
            )
        )
        
    def grad(self, inputs: tf.Tensor, targets: tf.Tensor) -> List:
        with tf.GradientTape() as g:
            error = self.loss(self(inputs), targets)
        
        return g.gradient(error, self.params)

    def optimize(self, grads: List[tf.Tensor], rate: float) -> None:
        opt = HyperParams.OPTIMIZER.value(learning_rate = rate)
        opt.apply_gradients(zip(grads, self.params))

In [46]:
data = Data(xdat,ydat)
data.partition()
data.to_tensor()
data.batch(HyperParams.BATCH_SIZE.value)

In [40]:
layer = Dense(4,2, tf.nn.relu)
layer(data.xtrn)

<tf.Tensor: id=385, shape=(105, 2), dtype=float32, numpy=
array([[ 7.011201 , 10.428638 ],
       [ 6.4876094,  9.6843405],
       [ 6.8864193, 10.159925 ],
       [ 7.027618 , 10.877469 ],
       [ 6.1899724,  9.349236 ],
       [ 5.899295 ,  9.236851 ],
       [ 6.634941 , 10.303792 ],
       [ 6.3556457,  9.915671 ],
       [ 6.5774   , 10.100556 ],
       [ 6.173556 ,  8.900406 ],
       [ 8.312613 , 11.592333 ],
       [ 7.7565374, 11.178372 ],
       [ 6.9239273, 10.374707 ],
       [ 7.5259185, 11.631507 ],
       [ 6.8750277, 10.256214 ],
       [ 6.7296205,  9.134917 ],
       [ 6.400929 , 10.494539 ],
       [ 6.0438766, 10.037334 ],
       [ 6.5685263, 10.559129 ],
       [ 6.467344 , 10.239203 ],
       [ 7.0504007, 10.68489  ],
       [ 6.1758404,  9.806005 ],
       [ 6.3006225, 10.074718 ],
       [ 7.143759 , 11.061439 ],
       [ 7.2477202, 10.420727 ],
       [ 7.7120805, 10.920318 ],
       [ 7.0209007, 10.290879 ],
       [ 7.687616 , 11.250873 ],
       [ 6.8305697

In [47]:
model = Chain([
    Dense(data.xtrn.shape[1], HyperParams.HIDDEN_NEURONS.value, HyperParams.ACTIVATION),
    Dense(HyperParams.HIDDEN_NEURONS.value, HyperParams.OUTPUT_NEURONS.value, HyperParams.NORMALIZER)
])

In [48]:
def accuracy(y, yhat):
    j = 0; correct = []
    for i in tf.argmax(y, 1):
        if i == tf.argmax(yhat[j]):
            correct.append(1)
        
        j += 1
    
    num = tf.cast(tf.reduce_sum(correct), dtype = tf.float32)
    den = tf.cast(y.shape[0], dtype = tf.float32)
    return num / den

In [49]:
epoch_trn_loss = []
epoch_tst_loss = []
epoch_trn_accy = []
epoch_tst_accy = []
for j in range(HyperParams.EPOCHS.value):
    trn_loss = []; trn_accy = []
    for i in range(len(data.xtrn_batches)):
        model.backward(data.xtrn_batches[i], data.ytrn_batches[i])
        ypred = model(data.xtrn_batches[i])
        trn_loss.append(model.loss(ypred, data.ytrn_batches[i]))
        trn_accy.append(accuracy(data.ytrn_batches[i], ypred))

    trn_err = tf.reduce_mean(trn_loss).numpy()
    trn_acy = tf.reduce_mean(trn_accy).numpy()

    tst_loss = []; tst_accy = []
    for i in range(len(data.xtst_batches)):
        ypred = model(data.xtst_batches[i])
        tst_loss.append(model.loss(ypred, data.ytst_batches[i]))
        tst_accy.append(accuracy(data.ytst_batches[i], ypred))
    
    tst_err = tf.reduce_mean(tst_loss).numpy()
    tst_acy = tf.reduce_mean(tst_accy).numpy()
    
    epoch_trn_loss.append(trn_err)
    epoch_tst_loss.append(tst_err)
    epoch_trn_accy.append(trn_acy)
    epoch_tst_accy.append(tst_acy)
    
    if j % 20 == 0:
        print("Epoch: {0:4d} \t Training Error: {1:.4f} \t Testing Error: {2:.4f} \t Accuracy Training: {3:.4f} \t Accuracy Testing: {4:.4f}".format(j, trn_err, tst_err, trn_acy, tst_acy))

Epoch:    0 	 Training Error: 9.2442 	 Testing Error: 6.6833 	 Accuracy Training: 0.3333 	 Accuracy Testing: 0.5238
Epoch:   20 	 Training Error: 6.3988 	 Testing Error: 4.7003 	 Accuracy Training: 0.3333 	 Accuracy Testing: 0.5238
Epoch:   40 	 Training Error: 3.5070 	 Testing Error: 2.5939 	 Accuracy Training: 0.3333 	 Accuracy Testing: 0.5238
Epoch:   60 	 Training Error: 1.7858 	 Testing Error: 1.3807 	 Accuracy Training: 0.4381 	 Accuracy Testing: 0.5238
Epoch:   80 	 Training Error: 1.3339 	 Testing Error: 1.0511 	 Accuracy Training: 0.4571 	 Accuracy Testing: 0.5397
Epoch:  100 	 Training Error: 0.9754 	 Testing Error: 0.7899 	 Accuracy Training: 0.4667 	 Accuracy Testing: 0.5238
Epoch:  120 	 Training Error: 0.7031 	 Testing Error: 0.6033 	 Accuracy Training: 0.5429 	 Accuracy Testing: 0.5873
Epoch:  140 	 Training Error: 0.5585 	 Testing Error: 0.5117 	 Accuracy Training: 0.7524 	 Accuracy Testing: 0.7778
Epoch:  160 	 Training Error: 0.4834 	 Testing Error: 0.4747 	 Accuracy 